In [ ]:
knitr::opts_chunk$set(
  collapse = TRUE,
  comment = "#>"
)

A *change point* is a structural shift in the data, such as a jump in mean or a change in trend.
Before conducting flood frequency analysis, we recommend splitting the data at change points to ensure each segment is approximately *trend-stationarity*.
The FFA framework provides two statistical tests for change point detection: 

- The Pettitt test, for sudden changes in the mean.
- The Mann-Kendall-Sneyers (MKS) test, for detecting changes in trend.

This vignette will demonstrate how these statistical tests can be used together to robustly identify change points in the data.

## Setup 

In [ ]:
library(ffaframework)

csv_path <- system.file("extdata", "Application_2.csv", package = "ffaframework")
df <- read.csv(csv_path)
df <- subset(df, !is.na(max)) # Remove missing values

head(df)

## The Pettitt Test

This rank-based test detects a single abrupt change in the median of a time series.
The null hypothesis assumes no change point.

Use the `eda_pettitt_test` function to perform the test. It requires two arguments: 

- `data`: numeric vector of streamflow measurements
- `years`: corresponding numeric vector of years

In [ ]:
pettitt_test <- eda_pettitt_test(df$max, df$year)

print(pettitt_test$p_value)

print(pettitt_test$change_year)

plot_pettitt_test(pettitt_test)

**Conclusion**: A p-value of 0 provides strong evidence of a change point in the year 1972.

## The MKS Test

The Mann-Kendall-Sneyers (MKS) test identifies trend changes in the data.

Use `eda_mks_test` with the same arguments as above.

In [ ]:
mks_test <- eda_mks_test(df$max, df$year)

print(mks_test$p_value)

print(mks_test$change_df$year)

plot_mks_test(mks_test)

**Conclusion**: At a p-value of 0.015, there is evidence of trend changes in 1960 and 1985.

**Note**: Since the MKS test can identify multiple change points, the reported p-value is determined using the *most significant* change point.

## Interpreting and Selecting Change Points

In this example, the Pettitt and MKS tests both suggest structural changes in the time series.

Consider the following guidelines when choosing where to split the data:

1. **Incorporate domain knowledge**. For example, if we knew that a dam was built in 1972, this would support the Pettitt test result.
2. **Avoid overpartitioning**. The Pettitt and MKS tests operate independently and may detect the same change point independently.
3. **Prioritize based on p-value**. Lower p-values indicate stronger evidence and should be given more weight.
4. **Conduct recursive analysis**. After splitting the series, consider reapplying the tests to each segment to detect secondary change points. For example, after splitting at 1972, reapply the MKS test to 1928–1971 and 1972–2020.